# Activity Coefficients in Electrolyte Solutions
_Mikael Lund_

In [ ]:
from __future__ import print_function
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import mdtraj as md
from math import sqrt, pi, exp
import os.path, os, sys, json, filecmp, copy
#plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir
print(workdir)

### Download and compile the MC software

In [ ]:
%%bash -s "$workdir"
%cd -q $1

echo 'fau_example(excess "./" excess.cpp)' > mc/CMakeLists.txt

# the following lines are for compilation on LUNARC
#module add GCC/5.4.0-2.26
#module load CMake/3.5.2
#export CXX=/sw/easybuild/software/Core/GCCcore/5.4.0/bin/g++
#export CC=/sw/easybuild/software/Core/GCCcore/5.4.0/bin/gcc

if [ ! -d "faunus/" ]; then
    git clone https://github.com/mlund/faunus.git
    cd faunus
    #git checkout a2220dc8f98606d2ceb42f14c6a08ec0723d8787
else
    cd faunus
fi

cmake . -DCMAKE_BUILD_TYPE=Release -DENABLE_APPROXMATH=on -DMYPLAYGROUND=$1/mc &>/dev/null
make excess -j4
%cd $1

### Simulation setup
Wall time, number of cores etc. Currently for the slurm system.

In [ ]:
%%writefile submit.sh
#!/bin/bash
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -t 03:00:00
../mc/excess > out

In [ ]:
salts = pd.Series(
    {
        'NaCl_2.1' : pd.Series( {'cation':'Na', 'anion':'Cl', 'ratio':1,
                           'activities':np.arange(0.1, 1.6, 0.1), 'catsigma':2.1, 'ansigma':2.1 } ),
        'NaCl_2.2' : pd.Series( {'cation':'Na', 'anion':'Cl', 'ratio':1,
                           'activities':np.arange(0.1, 1.6, 0.1), 'catsigma':2.2, 'ansigma':2.2 } ),
        'NaCl_2.3' : pd.Series( {'cation':'Na', 'anion':'Cl', 'ratio':1,
                           'activities':np.arange(0.1, 1.6, 0.1), 'catsigma':2.3, 'ansigma':2.3 } )
    }
)
salts['NaCl_2.3'].activities

In [ ]:
%cd -q $workdir

def mkinput():
    js = {
            "moleculelist": {
                "salt": { "Ninit": 20, "atomic": True, "atoms": d.cation+d.ratio*(' '+d.anion) }
            }, 
            "energy": {
                "nonbonded": { "coulomb": { "epsr": 80 } }
            }, 

            "moves": {
                "atomtranslate": { "salt": { "permol": True, "prob": 0.01 } }, 
                "atomgc": { "molecule": "salt" }
            }, 

            "system": {
                "mcloop"      : { "macro": 10, "micro": micro }, 
                "cuboid"      : { "len": 50 },
                "coulomb"     : { "epsr": 80 },
                "temperature" : 298.15
            }, 

            "atomlist": {
                "Na": { "q":  1.0, "r": catsigma, "eps":0.01, "dp": 70, "activity": activity }, 
                "Cl": { "q": -1.0, "r": ansigma, "eps":0.01, "dp": 70, "activity": activity }
            }
    }

    with open('excess.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))

# flow control:
equilibration = True   # if true, delete state file and start over
production    = True   # if true, start from present state file
slurm         = False  # if true, submit production runs to slurm cluster
override      = False   # if true, override existing files

for salt,d in salts.iteritems():
    for activity in d.activities:
        cation = d.cation
        anion  = d.anion
        ratio  = d.ratio
        catsigma=d.catsigma
        ansigma =d.ansigma
        
        pfx=salt+'-a'+str(activity)
        if not os.path.isdir(pfx):
            %mkdir -p $pfx
        else:
            if override==False:
                break
        %cd $pfx

        # equilibration run
        if equilibration:
            !rm -fR state
            micro=5000
            mkinput()
            !../mc/excess > eq 2>&1
            !rm -fR analysis_out.json

        # production run
        if production:
            micro=50000
            mkinput()
            if slurm:
                !sbatch ../submit.sh > /dev/null
            else:
                !../mc/excess > out 2>&1

        %cd -q ..

print('done.')

In [ ]:
%cd -q $workdir
import json

picklefile = 'data.p' 

if os.path.isfile( picklefile ):        # if pickle file exists, use that
    print('loading from saved pickle')
    sets = pd.read_pickle( picklefile )
else:
    print('loading from MC raw data')   # otherwise extract from MC output

    data = {}
    for salt,d in salts.iteritems():
        d['gamma_cation'] = []
        d['gamma_anion'] = []
        for activity in d.activities:
            pfx=salt+'-a'+str(activity)
        
            if os.path.isdir(pfx):
                %cd -q $pfx
                r = json.load( open('move_out.json') )['moves']['moves/atomgc']['atoms']
                d['gamma_cation'].append( np.array(r[d.cation]['gamma']) )
                d['gamma_anion'].append( np.array(r[d.anion]['gamma']) )
                %cd -q ..
    #salts.to_pickle( picklefile )
print(salts['NaCl_2.3'])

In [ ]:
# experimental data
exp = np.loadtxt('exp-nacl-coeff.csv', delimiter=',', skiprows=1, unpack=True)
plt.plot(exp[0], exp[1], 'ko', label='exp')

for salt,d in salts.iteritems():
    # mean ionic activity coefficient
    gamma = np.sqrt( np.array(d.gamma_cation) * np.array(d.gamma_anion) )
    
    # molar concentration = activity / gamma
    molarity = d.activities / gamma

    plt.plot( molarity, gamma, label='mc '+salt, lw=2)

plt.legend(loc=0, frameon=False)

plt.ylabel('$\gamma_{\pm}$')
plt.xlabel('molarity')
#plt.xlim((0.1,1.7))

In [ ]:
from scipy.optimize import newton

lB=7           # Bjerrum length (Å)
s=2*2.2        # sigma (Å)
eps=0.01 / 2.5 # epsilon (kT)

def el(r): return -lB/r                        # Coulomb pot.
def lj(r): return 4*eps*( (s/r)**12-(s/r)**6 ) # Lennard-Jones pot.
def u(r): return lj(r) + el(r)                 # combined pot.

r = np.linspace(0.64*s, 2*s, 100)

plt.plot(r, lj(r), 'r.', label='lj', markersize=2 )
plt.plot(r, el(r), label='el')
plt.plot(r, u(r), label='lj+el' )
plt.xlabel(u'$r$/Å')
plt.ylabel(u'$u(r)/k_BT$')
plt.legend(loc=0, frameon=False)

r0 = newton(u, x0=0.5*s) # when is u(r) zero?
plt.plot([r0], [0], 'go')
plt.annotate( '$r_0=$'+str('%.2f' % r0)+u' Å', xy=(rzero, 0), xytext=(r0+1, -1),
             arrowprops=dict(facecolor='green', width=2, headwidth=6, edgecolor='none') )